# Projet 3 - SSL pour la détection d'Anomalie - Groupe 12
Groupe 12 : Lise Catalogna, Julien Richaume, Romain Cargnello, Romain Laurié.

In [ ]:
import os
import csv
import random
import requests
import tarfile
import zipfile
import itertools
from itertools import product
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import Counter
import pandas as pd
import ast
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
   
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
#POUR IMPORTER DES PAQUETS DANS L'ENVIRONNEMENT IPYKERNEL JUPYTER NOTEBOOK
#import sys
#!"{sys.executable}" -m pip install scikit-learn

## Préparation des données

On doit charger les datasets depuis les sites web, qui sont dans des dossiers compréssés. Ainsi, on crée deux fonctions téléchargeants directement les fichiers et en les décompressant, une fonction pour chaque type de compression, zip et tar.

In [ ]:
def download_and_unzip(url, output_dir, zip_name="images.zip"):
    """
    Télécharge un fichier zip contenant des images et le décompresse dans un dossier.
    
    :param url: URL du fichier zip
    :param output_dir: Répertoire où les images seront décompressées
    :param zip_name: Nom temporaire pour le fichier zip téléchargé
    """
    # Download the zipfile on the wanted website
    print(f"Téléchargement du fichier depuis {url}...")
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(zip_name, "wb") as file:
            file.write(response.content)
        print("Téléchargement terminé.")
    else:
        print("Erreur lors du téléchargement.")
        return

    # Create the output repo if necessary
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Unzip the file
    print(f"Décompression dans {output_dir}...")
    with zipfile.ZipFile(zip_name, "r") as zip_ref:
        zip_ref.extractall(output_dir)
    print("Décompression terminée.")

    # Delete the temporary zip file
    os.remove(zip_name)
    print("Fichier zip temporaire supprimé.")


def download_and_extract_tar(url, output_dir, tar_name="images.tar.gz"):
    """
    Télécharge un fichier TAR contenant des images et le décompresse dans un dossier.
    
    :param url: URL du fichier TAR
    :param output_dir: Répertoire où les images seront décompressées
    :param tar_name: Nom temporaire pour le fichier TAR téléchargé
    """
    # Download the TAR file on the wanted website
    print(f"Téléchargement du fichier depuis {url}...")
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(tar_name, "wb") as file:
            file.write(response.content)
        print("Téléchargement terminé.")
    else:
        print("Erreur lors du téléchargement.")
        return

    # Create the output repo if necessary
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Untar the file
    print(f"Décompression dans {output_dir}...")
    if tarfile.is_tarfile(tar_name):
        with tarfile.open(tar_name, "r:*") as tar_ref:
            tar_ref.extractall(output_dir)
        print("Décompression terminée.")
    else:
        print(f"Le fichier {tar_name} n'est pas un fichier TAR valide.")
        return

    # Delete the temporary TAR file
    os.remove(tar_name)
    print("Fichier TAR temporaire supprimé.")

## Definir le répertoire de sortie   
output_dir = r"/home/rlaurie/Bureau/5A/HDDL/projet3/placeholder"
#output_dir = r"/home/catalogn/Documents/5A/HDDL/projets/projet3"
    
url = "https://zenodo.org/records/10459003/files/engine_wiring.zip?download=1" 
#download_and_unzip(url, output_dir)

url1 = "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937454-1629951595/capsule.tar.xz"
#download_and_extract_tar(url1, output_dir)

url2 = "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937545-1629951845/hazelnut.tar.xz"
#download_and_extract_tar(url2, output_dir)

url3 = "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938134-1629953256/toothbrush.tar.xz"
#download_and_extract_tar(url3, output_dir)

url4 = "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937370-1629951468/bottle.tar.xz"
#download_and_extract_tar(url4, output_dir)


In [ ]:
#On vérifie qu'on a bien chargé nos images en en affichant une
image = mpimg.imread('bottle/train/good/000.png')
image2 = mpimg.imread('capsule/train/good/000.png')
image3 = mpimg.imread('hazelnut/train/good/000.png')
plt.imshow(image3)
plt.axis('off')  # Optionnel : désactiver les axes
plt.show()

In [ ]:
#On fait la même chose en affichant une image de chaque jeu de données
images = [
    mpimg.imread('bottle/train/good/000.png'),
    mpimg.imread('capsule/train/good/000.png'),
    mpimg.imread('hazelnut/train/good/000.png'),
    mpimg.imread('toothbrush/train/good/000.png'),
    mpimg.imread('engine_wiring/train/good/0000.png')
]

plt.figure(figsize=(8, 4))

for i, img in enumerate(images):
    plt.subplot(1, len(images), i + 1)  # 1 ligne, n colonnes, position i+1
    plt.imshow(img)
    plt.title(f"Image {i+1}")
    plt.axis('off')

plt.tight_layout()
plt.show()

Maintenant que nos images sont bien chargés, on doit s'assurer qu'elles sont bien dans un même format. Or on a remarqué que lors du chargement il y a des différences, ainsi nous uniformisons tous cela grâce à la fonctions suivante.

Cette fonction permet à la fois de déplacer et de renommer les images, en effet on a remarqué qu'engine_wiring avait un nom différents avec xxxx.png au lieu de xxx.png, ainsi on a la renomme. De plus, pour simplifier l'accès aux données plus tard, on regroupe toutes les anomalies dans un dossier bad, ce qui permet d'uniformiser nos fonctions pour la suite.


In [ ]:
import os
import shutil
def move_and_rename_images(src_dir, dest_dir):
    """
    Déplace et renomme les images des sous-dossiers dans un seul dossier cible.
    
    :param src_dir: Chemin du dossier contenant les sous-dossiers
    :param dest_dir: Chemin du dossier cible
    """
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)  # Crée le dossier cible s'il n'existe pas

    counter = 1  # Compteur pour différencier les fichiers
    for root, _, files in os.walk(src_dir):  # Parcourt récursivement les sous-dossiers
        for file in files:
            if file.endswith(".png"):  # Vérifie que c'est une image .png
                # Chemin complet de l'image source
                src_path = os.path.join(root, file)
                
                # Nouveau nom avec compteur
                new_name = f"{counter}.png"
                dest_path = os.path.join(dest_dir, new_name)
                
                # Déplace et renomme l'image
                shutil.move(src_path, dest_path)
                print(f"Moved: {src_path} -> {dest_path}")
                
                counter += 1  # Incrémente le compteur

src_directory = "engine_wiring/test/bad"
dest_directory = "engine_wiring/test/bad" 
#move_and_rename_images(src_directory, dest_directory)

src_directory = "bottle/test/bad"
dest_directory = "bottle/test/bad" 
#move_and_rename_images(src_directory, dest_directory)

src_directory = "capsule/test/bad"
dest_directory = "capsule/test/bad" 
#move_and_rename_images(src_directory, dest_directory)

src_directory = "hazelnut/test/bad"
dest_directory = "hazelnut/test/bad" 
#move_and_rename_images(src_directory, dest_directory)

src_directory = "toothbrush/test/bad"
dest_directory = "toothbrush/test/bad" 
#move_and_rename_images(src_directory, dest_directory)

Maintenant que nos données sont uniformisées, on peut créer les Tenseurs et les DataLoaders encodant nos images.

Après plusieurs tests, nous avons remarqués, que normaliser nos données faisait baisser les précisions de nos modèles, ainsi nous avons décidé de ne pas les normaliser. Cependant nous redimmensions toutes. Pour cela, nous avons tout d'abord pris la taille standardisée (224,224), mais nous avons décidé de plutôt opter pour du (400,400), qui est la taille de la plus petit image de notre dataset. En effet, cerrtaines anomalies, par exemple pour le jeu de données "toothbrush" sont assez inifimes, ainsi en baissant encore plus la dimension de nos images nous avions peur que notre modèle n'arrive pas à distinguer les anomalies, étant devenues trop petites.

In [ ]:
data_dir = ['bottle', 'hazelnut', 'toothbrush', 'engine_wiring', 'capsule']
im_train = []
im_test = [[] for _ in range(len(data_dir))]
i=0
for data in data_dir:
    print("######")
    print(data)
    # Définir les transformations des images (normalisation, redimensionnement, etc.)
    transform = transforms.Compose([
        transforms.Resize((400, 400)),  # Redimensionner les images
        transforms.ToTensor(),  # Convertir les images en tenseurs
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation standard pour les modèles pré-entraînés
    ])
    
    # Charger le dataset en spécifiant le dossier train ou test
    train_dataset = datasets.ImageFolder(root=f"{data}/train", transform=transform)
    test_dataset = datasets.ImageFolder(root=f"{data}/test", transform=transform)

    # Créer des variables distinctes pour les DataLoaders
    exec(f"{data}_train_dataset = train_dataset")
    exec(f"{data}_test_dataset = test_dataset")
    
    # Créer les DataLoaders
    batch_size = 32
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Créer des variables distinctes pour les DataLoaders
    exec(f"{data}_train_loader = train_loader")
    exec(f"{data}_test_loader = test_loader")

    
    # Vérifier le nombre d'images par dataset
    print(f"Nombre d'images pour le train : {len(train_dataset)}")
    im_train.append(len(train_dataset))
    print(f"Nombre d'images pour le test : {len(test_dataset)}")
    
    # Classes
    print(f"Classes détectées dans train : {train_dataset.classes}")
    print(f"Classes détectées dans test : {test_dataset.classes}")

    # Récupérer les indices des classes pour chaque image
    class_counts = Counter(test_dataset.targets)
    
    # Obtenir les noms des classes
    class_names = test_dataset.classes
    
    # Afficher les résultats
    for class_idx, count in class_counts.items():
        print(f"Classe '{class_names[class_idx]}': {count} images")
        im_test[i].append(count)
    i+=1
    print(i)

Maintenant que nos DataLoaders sont crées, on regarde leur composition ci-dessus, mais aussi avec des histogrammes ci-dessous.

In [ ]:
# Liste des catégories et des données d'entraînement/test

categories = ['bottle', 'hazelnut', 'toothbrush', 'engine_wiring', 'capsule']

# Indices des catégories pour positionner les barres
indices = np.arange(len(categories))

# Largeur des barres
bar_width = 0.35
print(im_test)

# Calcul des segments pour chaque classe
test_class_1 = [counts[0] for counts in im_test]
test_class_2 = [counts[1] for counts in im_test]

# Création de l'histogramme
plt.figure(figsize=(10, 6))
plt.bar(indices, im_train, bar_width, label='Train', color='mediumorchid', alpha=0.7)
plt.bar(indices + bar_width, test_class_1, bar_width, label='Test - Bad', color='tomato', alpha=0.7)
plt.bar(indices + bar_width, test_class_2, bar_width, bottom=test_class_1, label='Test - Good', color='forestgreen', alpha=0.7)

# Ajout des étiquettes et des titres
plt.xlabel('Catégories', fontsize=12)
plt.ylabel('Nombre d\'images', fontsize=12)
plt.title('Répartition des images entre train et test par catégorie', fontsize=14)
plt.xticks(indices + bar_width / 2, categories, rotation=15, fontsize=10)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

On remarque plusieurs choses : 
- Pour "engine_wiring" il y a beaucoup plus d'images dans le test que dans le train ce qui pourrait peut être poser problème plus tard.
- Pour "toothbursh" il y a, aussi, peu de données tout court, que ça soit pour le train comme pour le test, ainsi notre modèle pourra peut être surapprendre sur nos images, ce qui pourra être un problème. De plus, la répartition est d'environ 60-40, ce qui est inférieur à ce qu'on a habituellement, ce qui pourrait peut être également poser problème plus tard.
- Pour le reste il y a plus de données d'entraîenements que de tests, et plus d'anomalies que de bonnes images dans le jeu de données de tests. 

## Définition des modèles

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_channels=3, latent_dim=128):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1),  # 32x32 -> 16x16
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # 16x16 -> 8x8
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # 8x8 -> 4x4
            nn.ReLU(),
            nn.Conv2d(256, latent_dim, kernel_size=4, stride=2, padding=1),  # 4x4 -> 2x2
            nn.ReLU(),
        )

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim=128, out_channels=3):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 256, kernel_size=4, stride=2, padding=1),  # 2x2 -> 4x4
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 4x4 -> 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 8x8 -> 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(64, out_channels, kernel_size=4, stride=2, padding=1),  # 16x16 -> 32x32
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.decoder(x)

On définit désormais la fonction qui servira pour entraîner nos modèles, elle servira aussi à afficher les courbes de pertes, pour vérifier le bon apprentissage de nos modèles.

In [ ]:
def train_ssl_model(model, 
                    train_loader, 
                    test_loader, 
                    criterion,
                    optimizer,
                    device='cuda',
                    epochs=5,
                    show_plot=False):
    # Initialiser les listes pour stocker les pertes
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.to(device)
        model.train()
        total_train_loss = 0
        for images, _ in train_loader:
            images = images.to(device)
            optimizer.zero_grad()
            output, _ = model(images)
            loss = criterion(output, images)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)  # Stocker la perte d'entraînement

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for images, _ in test_loader:
                images = images.to(device)
                output, _ = model(images)
                val_loss = criterion(output, images)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(test_loader)
        val_losses.append(avg_val_loss)  # Stocker la perte de validation

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")
    
    if show_plot:
        # Tracer les courbes de pertes
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, epochs + 1), train_losses, label='Train Loss', marker='o')
        plt.plot(range(1, epochs + 1), val_losses, label='Validation Loss', marker='o')
        plt.xlabel('Époque', fontsize=12)
        plt.ylabel('Loss', fontsize=12)
        plt.title('Courbes de Training et Validation Loss', fontsize=14)
        plt.legend(fontsize=12)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    return model.encoder, avg_val_loss, model

In [ ]:
# Visualiser la recpnstruction d'images aléatoires dans le dataset de test
def visualize_reconstructions(model, data_loader, device, num_images=5):
    model.eval()
    
    dataset = list(data_loader.dataset)   # Pour convertir les DataLoader en un échantillon aléatoire d'image
    random_indices = random.sample(range(len(dataset)), num_images) # Sélection aléatoire des images `num_images` du jeu de données
    random_images = [dataset[i][0] for i in random_indices]  # Extraire uniquement les images, en ignorant les étiquettes

    images = torch.stack(random_images)   # Empiler les images dans un lot
    print(device)
    images = images.to(device)
    
    # On passe les images dans le modèle mis en argument
    with torch.no_grad():
        reconstructed_images, perturbed_images = model(images)
    
    # On remet les images sur le CPU pour la visualization
    images = images.cpu()
    reconstructed_images = reconstructed_images.cpu()
    perturbed_images = perturbed_images.cpu()
    
    # Afficher l'image reçue par le modèle, la vérité terrain et l'image reconstruite
    fig, axes = plt.subplots(num_images, 3, figsize=(10, num_images * 4))
    for i in range(num_images):
        # Grayscale input
        axes[i, 0].imshow(perturbed_images[i].permute(1, 2, 0).squeeze(), cmap='gray')
        axes[i, 0].set_title("Input du modèle")
        axes[i, 0].axis('off')
        
        # Ground truth (original RGB image)
        axes[i, 1].imshow(images[i].permute(1, 2, 0))
        axes[i, 1].set_title("Vérité Terrain")
        axes[i, 1].axis('off')
        
        # Colorized output from the model
        axes[i, 2].imshow(reconstructed_images[i].permute(1, 2, 0))
        axes[i, 2].set_title("Image Reconstruite")
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

## Méthode 1 : COLORIZATION

On commence avec le premier modèle : Colorization.

Le modèle de colorisation apprend à convertir des images en niveaux de gris en images colorées. Cette tâche impose au modèle de capturer des caractéristiques de haut niveau dans les images, ce qui est utile pour détecter des différences subtiles ou des anomalies. Plus précisement, on l'a choisi car dans le cas de la détection d'anomalie, une mauvaise reconstruction des couleurs indique que le modèle ne reconnaît pas certaines caractéristiques, ce qui est un bon indicateur d'une anomalie.

In [ ]:
class ColorizationModel(nn.Module):
    def __init__(self, latent_dim=128):
        super(ColorizationModel, self).__init__()
        self.encoder = Encoder(latent_dim=latent_dim, in_channels=1)  # Input grayscale
        self.decoder = Decoder(latent_dim=latent_dim, out_channels=3)  # Predict RGB

    def forward(self, x):
        grayscale_x = transforms.Grayscale()(x)  # Convert RGB to Grayscale
        z = self.encoder(grayscale_x)
        return self.decoder(z), grayscale_x

In [ ]:
colorization_model = ColorizationModel(latent_dim=128)
colorization_encoder, val, colorization_model = train_ssl_model(colorization_model, 
                                                                hazelnut_train_loader, 
                                                                hazelnut_test_loader, 
                                                                criterion=nn.MSELoss(), 
                                                                optimizer=optim.Adam(colorization_model.parameters(), lr=0.001),
                                                                device=device,
                                                                epochs=20
                                                                )

In [ ]:
# Visualiser la colorisation sur des images test aléatoires
visualize_reconstructions(colorization_model, hazelnut_test_loader, device)

On remarque qu'avec un modèle pas optimisé et peu d'époques on arrive bien à reconstruire les anomalies, surtout les trous dans la noisette. Cependant les images sont tout de même assez floues, ce qui pose problème pour détecter des coupures ou des rayures sur la noisette.

## Méthode 2 : INPAINTING

Le 2ème modèle utilisé est l'inpainting. C'est une tâche qui consiste à reconstruire des parties manquantes dans une image, qui sont masquées par un carré noir (masque) intégré aux images dans l'entraînement. On a choisi ce modèle est choisi pour sa capacité à apprendre des représentations visuelles en explorant les relations contextuelles entre différentes régions de l'image.

Si un modèle est entraîné à reconstruire des images partielles (avec des masques), les anomalies deviendront plus visibles par des reconstructions moins précises.

In [ ]:
class InpaintingModel(nn.Module):
    def __init__(self, latent_dim=128, mask_size=8):
        super(InpaintingModel, self).__init__()
        self.encoder = Encoder(latent_dim=latent_dim)
        self.decoder = Decoder(latent_dim=latent_dim)
        self.mask_size = mask_size
    
    #forward method
    def forward(self, x):
        x_masked = self.apply_mask(x)
        z = self.encoder(x_masked)
        return self.decoder(z), x_masked

    #mask method
    def apply_mask(self, x):
        masked_x = x.clone()

        for i in range(masked_x.size(0)):
            ul_x = np.random.randint(0, x.size(2) - self.mask_size + 1) #Randomly sample the x coordinate of the upper left corner
            ul_y = np.random.randint(0, x.size(3) - self.mask_size + 1) #Randomly sample the y coordinate of the upper left corner
            masked_x[i, :, ul_x:ul_x+self.mask_size, ul_y:ul_y+self.mask_size] = 0

        return masked_x

In [ ]:
inpainting_model = InpaintingModel(mask_size=36) 
inpainting_encoder, val, inpainting_model = train_ssl_model(inpainting_model,
                                                            capsule_train_loader, 
                                                            capsule_test_loader, 
                                                            criterion=nn.MSELoss(), 
                                                            optimizer=optim.Adam(inpainting_model.parameters(), lr=0.001),
                                                            device=device,
                                                            epochs=20
                                                            )

In [ ]:
visualize_reconstructions(inpainting_model, capsule_test_loader, device=device, num_images=5)

On remarque plus ou moins la même chose qu'avec le modèle de colorization et le dataset des noisettes. On a du mal à bien reconstruire le chiffre sur la capsule et on arrive à peu près bien à reconstruire les capsules, et les masques semblent bien perturbées un peu l'images ce qui force le modèle à se concentrer sur sa reconstruction.

## Méthode 3 : Masked Autoencoder

Pour finir, on a choisir de partir avec un Masked AutoEncoder. Les MAEs masquent une proportion significative des pixels d'entrée et apprennent à reconstruire ces pixels masqués. On a choisi ce modèle, car cela le pousse à comprendre globalement les structures et les textures des images. Ainsi la reconstruction des régions masquées d'une image anormale est souvent moins précise, ce qui fait des MAEs un choix adapté.

In [ ]:
class MaskedAutoencoderModel(nn.Module):
    def __init__(self, latent_dim=128, mask_ratio=1/16):
        super(MaskedAutoencoderModel, self).__init__()
        self.encoder = Encoder(latent_dim=latent_dim)
        self.decoder = Decoder(latent_dim=latent_dim)
        self.mask_ratio = mask_ratio

    def forward(self, x):
        x_masked = self.apply_mask(x)
        z = self.encoder(x_masked)
        return self.decoder(z), x_masked
    
    def apply_mask(self, x):
        x_masked = x.clone()
        mask = torch.rand_like(x[:, 0, :, :]) < self.mask_ratio
        mask = mask.unsqueeze(1).repeat(1, x.size(1), 1, 1)
        x_masked[mask] = 0
        return x_masked

In [ ]:
mae_model = MaskedAutoencoderModel(latent_dim=128, mask_ratio=2/16)
mae_encoder, val, mae_model = train_ssl_model(mae_model,
                                              engine_wiring_train_loader, 
                                              engine_wiring_test_loader, 
                                              criterion=nn.MSELoss(), 
                                              optimizer=optim.Adam(mae_model.parameters(), lr=0.001),
                                              device=device,
                                              epochs=20
                                              )

In [ ]:
visualize_reconstructions(mae_model, engine_wiring_test_loader, device=device, num_images=10)

De même qu'avant, on reconnait bien les câbles mais on ne distingue pas parfaitement les couleurs notamment le vert et le bleu.

## Fine Tunning

Maintenant que nos modèles sont définis, on voit que les résultats sont largement améliorables. Pour cela, on a décider de finetuner nos hyperparamètres en dehors de ce notebook sur le serveur de calcul du GMM, ici vous pourrez retrouver les fonctions de GridSearch que nous avons utilisé pour cela.

In [ ]:
#On a modifié la fonction d'entraînement des modèles pour qu'elles n'affiche plus les courbes
#Cependant elle intégre désormais le fais d'écrire les résultats dans un fichier excel toutes les 5 époques.

def train_ssl_model_csv(model, 
                    train_loader, 
                    test_loader, 
                    criterion,
                    optimizer,
                    device='cuda',
                    epochs=5,
                    model_name="Model",
                    dataset_name="Dataset",
                    param_dict=None,
                    save_path="results.csv"):
    # Initialiser les listes pour stocker les pertes
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.to(device)
        model.train()
        total_train_loss = 0
        for images, _ in train_loader:
            images = images.to(device)
            optimizer.zero_grad()
            output, _ = model(images)
            loss = criterion(output, images)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)  # Stocker la perte d'entraînement

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for images, _ in test_loader:
                images = images.to(device)
                output, _ = model(images)
                val_loss = criterion(output, images)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(test_loader)
        val_losses.append(avg_val_loss)  # Stocker la perte de validation

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")

        # Sauvegarde dans un fichier CSV toutes les 5 époques
        if (epoch + 1) % 5 == 0:
            with open(save_path, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([
                    model_name, dataset_name, epoch + 1, 
                    param_dict, avg_train_loss, avg_val_loss
                ])

    return model.encoder, avg_val_loss


# On utilise cette fonction pour pouvoir optimiser les hyperparamètres :
# de mask_ratio pour Inpainting 
# de mask_size pour  MAE

def get_model_specific_param_grid(model_name, param_grid):
    param_grid = param_grid.copy()  # Créer une copie pour éviter d'affecter l'original
    if model_name != "MaskedAutoencoder":
        param_grid.pop('mask_ratio', None)  # Retirer 'mask_ratio' si non pertinent
    if model_name != "Inpainting":
        param_grid.pop('mask_size', None)  # Retirer 'mask_size' si non pertinent
    return param_grid


def grid_search_ssl(models, datasets, param_grid, criterion, device='cuda', epochs=5, save_path="results.csv"):
    """
    Recherche en GridSearch de modèles d'apprentissage auto-supervisé sur plusieurs ensembles de données.

    Args :
        models (dict) : Dictionnaire de modèles avec leurs noms comme clés.
        datasets (dict) : Dictionnaire avec les noms des ensembles de données comme clés et les tuples (train_loader, test_loader) comme valeurs.
        param_grid (dict) : Grille d'hyperparamètres (les clés sont des hyperparamètres, les valeurs sont des listes de valeurs possibles).
        criterion : Fonction de perte.
        device (str) : Dispositif à utiliser ('cuda' ou 'cpu').
        epochs (int) : Nombre d'époques à entraîner.
        save_path (str) : Chemin d'accès au fichier pour l'enregistrement des résultats.

    Retourne :
        results (dict) : Meilleurs paramètres et perte de validation correspondante pour chaque modèle et ensemble de données.
    """
    results = {}

    # Générer l'en-tête du fichier CSV
    with open(save_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Model", "Dataset", "Epoch", "Parameters", "Train Loss", "Validation Loss"])

    # Générer toutes les combinaisons d'hyperparamètres
    param_combinations = list(product(*param_grid.values()))

    for model_name, model_class in models.items():
        print(f"==== Grid Search for Model: {model_name} ====")
        results[model_name] = {}

        for dataset_name, (train_loader, test_loader) in datasets.items():
            print(f"  Dataset: {dataset_name}")
            best_loss = float('inf')
            best_params = None

            # Obtenir le param_grid spécifique au modèle
            param_grid = get_model_specific_param_grid(model_name, param_grid)

            # Générer toutes les combinaisons de paramètres pertinentes
            param_combinations = list(itertools.product(*param_grid.values()))

            for params in param_combinations:                
                # Créer le modèle avec les hyperparamètres actuels
                param_dict = dict(zip(param_grid.keys(), params))
                model_params = {k: v for k, v in param_dict.items() if k not in ['lr']}
                if model_name != "MaskedAutoencoder" :
                    model_params = {k: v for k, v in model_params.items() if k not in ['mask_ratio']}
                if model_name != "Inpainting" :
                    model_params = {k: v for k, v in model_params.items() if k not in ['mask_size']}
                # Afficher la combinaison de paramètres testée
                print(f"Testing parameters: {param_dict}")
                model = model_class(**model_params).to(device)

                optimizer = torch.optim.Adam(model.parameters(), lr=param_dict['lr'])

                # Entraîner le modèle
                trained_encoder, val_loss = train_ssl_model_csv(
                    model,
                    train_loader,
                    test_loader,
                    criterion,
                    optimizer,
                    device=device,
                    epochs=epochs,
                    model_name=model_name,
                    dataset_name=dataset_name,
                    param_dict=param_dict,
                    save_path=save_path
                )

                # Vérifier si la configuration actuelle est la meilleure
                if val_loss < best_loss:
                    best_loss = val_loss
                    best_params = param_dict

            # Stocker les meilleurs paramètres et les pertes
                results[model_name][dataset_name] = {
                'best_params': best_params,
                'best_val_loss': best_loss
            }

            # Ajouter les meilleurs résultats au CSV
            with open(save_path, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([
                    model_name, dataset_name, "Best", best_params, "-", best_loss
                ])

            print(f"    Best Params: {best_params}, Best Val Loss: {best_loss:.4f}")

    return results

In [ ]:
models = {
    'MaskedAutoencoder': lambda **params: MaskedAutoencoderModel(**params),
    'Colorization': lambda **params: ColorizationModel(**params),
    'Inpainting': lambda **params: InpaintingModel(**params),
}

datasets = {
    'bottle': (bottle_train_loader, bottle_test_loader),
    'hazelnut': (hazelnut_train_loader, hazelnut_test_loader),
    'toothbrush': (toothbrush_train_loader, toothbrush_test_loader),
    'engine_wiring': (engine_wiring_train_loader, engine_wiring_test_loader),
    'capsule': (capsule_train_loader, capsule_test_loader),
}


param_grid = {
    'lr': [0.01, 0.005, 0.001, 0.0001],
    'latent_dim': [64, 128, 256],
    'mask_size': [16, 32, 64],  # Pour InpaintingModel uniquement
    'mask_ratio': [1/16, 1/8, 1/4],  # Pour MaskedAutoencoderModel uniquement
}

criterion = nn.MSELoss()
#results = grid_search_ssl(models, datasets, param_grid, criterion, device, epochs=2)
#print(results)

Après le code pour sélectionner nos hyperparamètres, on les extrait du fichier excel.

In [ ]:
def extract_hyperparams_from_excel(data):
    results = []
    for _, row in data.iterrows():
        model = row['Model']  # Remplace par le nom exact de la colonne "Modèle"
        dataset = row['Dataset']  # Remplace par le nom exact de la colonne "Dataset"
        epochs = row['Epoch'] # Pour repérer la ligne où l'on stocker les meilleurs hyperparams
        best_hyperparams = row['Parameters']  # Remplace par la colonne contenant {'lr': ..., 'latent_dim': ...}
        
        # Convertir les hyperparamètres en dictionnaire Python
        if epochs == "Best":
            hyperparams = ast.literal_eval(best_hyperparams)
        
            results.append({
                'Modèle': model,
                'Dataset': dataset,
                'Hyperparamètres': hyperparams
            })
    
    return results

colorization_path = '/home/rlaurie/Bureau/5A/HDDL/projet3/results_color.ods'
data_color = pd.read_excel(colorization_path, engine="odf")
print(data_color.head())
extracted_color = extract_hyperparams_from_excel(data_color)

# Exemple d'utilisation des résultats
for entry in extracted_color:
    print(f"Modèle : {entry['Modèle']}")
    print(f"Dataset : {entry['Dataset']}")
    print(f"Hyperparamètres : {entry['Hyperparamètres']}")
    print("-" * 30)


inpainting_path = '/home/rlaurie/Bureau/5A/HDDL/projet3/results_inpainting.ods'
data_inpaint = pd.read_excel(inpainting_path, engine="odf")
print(data_inpaint.head())
extracted_inpaint = extract_hyperparams_from_excel(data_inpaint)

# Exemple d'utilisation des résultats
for entry in extracted_inpaint:
    print(f"Modèle : {entry['Modèle']}")
    print(f"Dataset : {entry['Dataset']}")
    print(f"Hyperparamètres : {entry['Hyperparamètres']}")
    print("-" * 30)


mae_path = '/home/rlaurie/Bureau/5A/HDDL/projet3/results_mae.ods'
data_mae = pd.read_excel(mae_path, engine="odf")
print(data_mae.head())
extracted_mae = extract_hyperparams_from_excel(data_mae)

# Exemple d'utilisation des résultats
for entry in extracted_mae:
    print(f"Modèle : {entry['Modèle']}")
    print(f"Dataset : {entry['Dataset']}")
    print(f"Hyperparamètres : {entry['Hyperparamètres']}")
    print("-" * 30)



## Evaluation de nos résultats avec des courbes ROC et métrique AUROC 

Maintenant que nous avons nos hyperparamètres optimisés pour chaque combinaison de modèles et de datasets, nous pouvons passer à l'éévaluation de leur performance, avec des courbes ROC et métrique AUROC.

### Threshold (Seuil)

Tout d'abord pour les courbes ROC, nous deons définir un seuil. En effet, ce seuil servira à classifier les sorties du modèles. En effet, on le construit de telle manière que 90% des images du jeu d'entraînement soient reconnues comment normale et on regarde si la précision de notre recostruction est au-dessus ou en-dessous de ce seuil pour définir s'il y a une anomalie ou non.

In [ ]:
def compute_thresholds_for_multiple_models(models, train_loader, criterion, device='cuda', normal_ratio=0.9):
    """
    Calculer les seuils d'anomalie pour plusieurs modèles en utilisant l'ensemble d'entraînement.
    Args:
        models (dict): Dictionnaire des modèles avec leurs noms comme clés.
        train_loader: DataLoader pour l'ensemble d'entraînement.
        criterion: Fonction de perte pour calculer les scores d'anomalie.
        device (str): Appareil à utiliser ('cuda' ou 'cpu').
        normal_ratio (float): Ratio d'échantillons à classer comme "normaux" (par défaut 0.9, soit 90%).
    Returns:
        dict: Seuils calculés pour chaque modèle.
    """
    thresholds = {}

    for model_name, model in models.items():
        model.to(device)
        model.eval()
    
        anomaly_scores = []
    
        with torch.no_grad():
            for images, _ in train_loader:
                images = images.to(device)
                output, _ = model(images)
                # Calculer la perte comme score d'anomalie
                loss = torch.mean((output - images) ** 2, dim=(1, 2, 3))
                anomaly_scores.extend(loss.cpu().numpy())
    
        # Conversion en array
        anomaly_scores = np.array(anomaly_scores)
    
        # Calculer le seuil au 90e centile des scores d'anomalie
        threshold = np.percentile(anomaly_scores, normal_ratio * 100)
        thresholds[model_name] = threshold
        print(f"Threshold for {model_name} (90% normal): {threshold:.4f}")
    
    return thresholds

### ROC, AUROC, Matrice de Confusion et métrique de Précision, Recall et F1-Score

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_anomaly_detection_with_threshold_multiple_models(models, test_loader, dataset_name, criterion, thresholds, device='cuda'):
    """
    Évaluer la détection d'anomalies pour plusieurs modèles et tracer les courbes ROC sur le même graphique.
    Args :
        models (dict) : Dictionnaire de modèles avec leurs noms comme clés.
        test_loader : DataLoader pour le jeu de données de test.
        dataset_name (str) : Nom du jeu de données (pour le titre du graphique).
        criterion : Fonction de perte pour calculer les scores d'anomalie.
        thresholds (dict) : Dictionnaire des seuils pour chaque modèle.
        device (str) : Périphérique à utiliser (« cuda » ou « cpu »).
    Retourne :
        avg_roc_auc : Moyenne de l'AUROC sur l'ensemble des modèles.
    """
    roc_data = {}  # Stocker le FPR, le TPR et l'AUC pour chaque modèle
    all_predictions = []
    all_ground_truth = []
    
    # Evaluate each model
    for model_name, model in models.items():
        model.to(device)
        model.eval()
        
        anomaly_scores = []
        ground_truth = []
        predictions = []

        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                output, _ = model(images)
                loss = torch.mean((output - images) ** 2, dim=(1, 2, 3))
                anomaly_scores.extend(loss.cpu().numpy())
                
                # Tracer les étiquettes multi-classes en binaire (en supposant que 0 = bad, autre = good)
                binary_labels = (labels != 1).int()  # 0 -> bad, 1 -> good
                ground_truth.extend(binary_labels.cpu().numpy())

        # Convertir en tableaux numpy
        anomaly_scores = np.array(anomaly_scores)
        ground_truth = np.array(ground_truth)

        # Appliquer le seuil spécifique au modèle pour générer des prédictions
        threshold = thresholds.get(model_name, 0.05)   # Valeur par défaut de 0,05 si le seuil n'est pas fourni
        preds = (anomaly_scores > threshold).astype(int)  # 0 si anomalie, 1 si normal
        predictions.extend(preds)

        # Calcul de la courbe ROC et de l'AUROC
        fpr, tpr, thresholds_roc = roc_curve(ground_truth, anomaly_scores)
        roc_auc = auc(fpr, tpr)
        
        # Stocker les résultats
        roc_data[model_name] = {'fpr': fpr, 'tpr': tpr, 'roc_auc': roc_auc}

        # Calcul de la précision, du rappel et du score F1
        precision = precision_score(ground_truth, preds)
        recall = recall_score(ground_truth, preds)
        f1 = f1_score(ground_truth, preds)

        # Affichage de la matrice de confusion
        cm = confusion_matrix(ground_truth, preds)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Anomalie', 'Normal'], yticklabels=['Anomalie', 'Normal'])
        plt.title(f'Matrice de Confusion pour {model_name}')
        plt.xlabel('Prédictions')
        plt.ylabel('Véritables Labels')
        plt.show()

        # Affichage des métriques de précision, de rappel et de score F1
        print(f"{model_name} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"{model_name} - Confusion Matrix:\n{cm}\n")

                # Recueillir les prédictions et la vérité de terrain pour une évaluation globale
        all_predictions.extend(preds)
        all_ground_truth.extend(ground_truth)

    # Calculer la courbe ROC moyenne
    all_fpr = np.unique(np.concatenate([roc_data[m]['fpr'] for m in roc_data]))
    mean_tpr = np.zeros_like(all_fpr)

    for model_name in roc_data:
        mean_tpr += np.interp(all_fpr, roc_data[model_name]['fpr'], roc_data[model_name]['tpr'])

    mean_tpr /= len(models)
    mean_roc_auc = auc(all_fpr, mean_tpr)

    # Tracer toutes les courbes ROC
    plt.figure(figsize=(8, 6))
    for model_name in roc_data:
        plt.plot(
            roc_data[model_name]['fpr'], 
            roc_data[model_name]['tpr'], 
            lw=2, label=f"{model_name} (AUC = {roc_data[model_name]['roc_auc']:.2f})"
        )

    # Tracer la courbe ROC moyenne
    plt.plot(
        all_fpr, 
        mean_tpr, 
        color='violet', 
        lw=2, linestyle='-', 
        label=f"Mean ROC (AUC = {mean_roc_auc:.2f})"
    )

    # Ligne diagonale pour référence
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title(f'ROC Curves for Multiple Models on {dataset_name}', fontsize=14)
    plt.legend(loc="lower right", fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return mean_roc_auc


### Fonction pour tout exécuter en 1 fois

Comme chaque modèle doit être entrainé sur chaque dataset, cela peut arriver de se perdre et d'entraîner le modèle sur un mauvais dataset, alors nous avons tous rassemblé en une fonction qui prend en entrée les DataLoaders et les hyperparamètes associés à ce dataset et qui ressort les courbes ROC et le score AUROC

In [ ]:
def training_threshold_roc_curves_auroc_confusion(train_loader, test_loader, dataset_name, device, 
                                         epochs=20, 
                                         models_hyperparams=None,
                                         save_dir='/home/rlaurie/Bureau/5A/HDDL/projet3/poids'):
    """
    Entraînement des modèles, calcul des seuils, évaluation des courbes ROC et des scores AUROC, affichage des matrices de confusions et calcul des métrqiues de Précision de Recall et de score F1
    
    Args :
        train_loader : DataLoader pour l'ensemble d'entraînement.
        test_loader : DataLoader pour l'ensemble de tests.
        dataset_name : Nom de l'ensemble de données pour le tracé.
        epochs : Nombre d'époques d'apprentissage.
        models_hyperparams : Dictionnaire contenant des hyperparamètres spécifiques au modèle, tels que 
                             latent_dim, lr et mask_size.
                             Exemple de format :
                             {
                                Inpainting » : {“latent_dim” : 128, “mask_size” : 36, 'lr' : 0.001},
                                Colorization' : {'latent_dim' : 128, 'lr' : 0.001},
                                MaskedAutoencoder' : {'latent_dim' : 128, 'mask_ratio' : 0.125, 'lr' : 0.001}
                             }
        save_dir : Répertoire où les modèles seront sauvegardés.
    """
    print(device)
    
    # Initialiser les modèles avec des hyperparamètres spécifiques
    inpainting_model = InpaintingModel(mask_size=models_hyperparams['Inpainting']['mask_size'])
    colorization_model = ColorizationModel(latent_dim=models_hyperparams['Colorization']['latent_dim'])
    mae_model = MaskedAutoencoderModel(latent_dim=models_hyperparams['MaskedAutoencoder']['latent_dim'], 
                                       mask_ratio=models_hyperparams['MaskedAutoencoder']['mask_ratio'])

    print("####  Inpainting Model Training  #####")
    inpainting_encoder, val_loss, inpaintin_model = train_ssl_model(inpainting_model,
                                         train_loader, 
                                         test_loader, 
                                         criterion=nn.MSELoss(), 
                                         optimizer=optim.Adam(inpainting_model.parameters(), 
                                                              lr=models_hyperparams['Inpainting']['lr']),
                                         device=device,
                                         epochs=epochs,
                                         show_plot=True)

    print("####  Colorization Model Training  #####")
    colorization_encoder, val_loss, colorization_model = train_ssl_model(colorization_model, 
                                           train_loader, 
                                           test_loader, 
                                           criterion=nn.MSELoss(), 
                                           optimizer=optim.Adam(colorization_model.parameters(), 
                                                                lr=models_hyperparams['Colorization']['lr']),
                                           device=device,
                                           epochs=epochs,
                                           show_plot=True)

    print("####  Masked AutoEncoder Model Training  #####")
    mae_encoder, val_loss, mae_model = train_ssl_model(mae_model,
                                  train_loader, 
                                  test_loader, 
                                  criterion=nn.MSELoss(), 
                                  optimizer=optim.Adam(mae_model.parameters(), 
                                                       lr=models_hyperparams['MaskedAutoencoder']['lr']),
                                  device=device,
                                  epochs=epochs,
                                  show_plot=True)

    # Pour s'assurer que le répertoire de sauvegarde existe
    os.makedirs(save_dir, exist_ok=True)

    # Sauvegarder les modèles
    print("####  Saving Models  #####")
    torch.save(inpainting_model.state_dict(), f"{save_dir}/{dataset_name}_inpainting_model.pth")
    torch.save(colorization_model.state_dict(), f"{save_dir}/{dataset_name}_colorization_model.pth")
    torch.save(mae_model.state_dict(), f"{save_dir}/{dataset_name}_mae_model.pth")

    models = {
        'Inpainting': inpainting_model,
        'Colorization': colorization_model,
        'Masked Autoencoder': mae_model
    }

    print("####  Thresholds computations  #####")
    # Calcul des seuils pour tous les modèles
    thresholds = compute_thresholds_for_multiple_models(
        models=models, 
        train_loader=train_loader, 
        criterion=nn.MSELoss(), 
        device=device
    )
    
    # Évaluer et tracer les courbes ROC mais aussi affichage de la matrice de confusion et des métriques de Précision, Recall et F1 Score
    mean_roc_auc = evaluate_anomaly_detection_with_threshold_multiple_models(
        models=models, 
        test_loader=test_loader,
        dataset_name=dataset_name,
        criterion=nn.MSELoss(), 
        thresholds=thresholds, 
        device=device
    )
    return inpainting_model, colorization_model, mae_model

### Hazelnut Dataset

In [ ]:
# Filtrer pour "hazelnut" et reformater
hazelnut_mae = next((entry for entry in extracted_mae if entry['Dataset'] == 'hazelnut'), None)
formatted_hazelnut_mae = {hazelnut_mae['Modèle']: hazelnut_mae['Hyperparamètres']}
print(formatted_hazelnut_mae)

hazelnut_color = next((entry for entry in extracted_color if entry['Dataset'] == 'hazelnut'), None)
formatted_hazelnut_color = {hazelnut_color['Modèle']: hazelnut_color['Hyperparamètres']}
print(formatted_hazelnut_color)

hazelnut_inpaint = next((entry for entry in extracted_inpaint if entry['Dataset'] == 'hazelnut'), None)
formatted_hazelnut_inpaint= {hazelnut_inpaint['Modèle']: hazelnut_inpaint['Hyperparamètres']}
print(formatted_hazelnut_inpaint)

models_hyperparams_hazelnut = {**formatted_hazelnut_inpaint, **formatted_hazelnut_color, **formatted_hazelnut_mae}

In [ ]:
inpainting_model, colorization_model, mae_model = training_threshold_roc_curves_auroc_confusion(train_loader = hazelnut_train_loader, 
                                                                                                test_loader = hazelnut_test_loader,
                                                                                                dataset_name = "Hazelnut Dataset", 
                                                                                                device='cuda', 
                                                                                                epochs=40, 
                                                                                                models_hyperparams=models_hyperparams_hazelnut)

In [ ]:
# Visualiser la colorisation sur des images test aléatoires
visualize_reconstructions(inpainting_model, hazelnut_test_loader, device)
visualize_reconstructions(colorization_model, hazelnut_test_loader, device)
visualize_reconstructions(mae_model, hazelnut_test_loader, device)

### Bottle Dataset

In [ ]:
# Filtrer pour "bottle" et reformater
bottle_mae = next((entry for entry in extracted_mae if entry['Dataset'] == 'bottle'), None)
formatted_bottle_mae = {bottle_mae['Modèle']: bottle_mae['Hyperparamètres']}
print(formatted_bottle_mae)

bottle_color = next((entry for entry in extracted_color if entry['Dataset'] == 'bottle'), None)
formatted_bottle_color = {bottle_color['Modèle']: bottle_color['Hyperparamètres']}
print(formatted_bottle_color)

bottle_inpaint = next((entry for entry in extracted_inpaint if entry['Dataset'] == 'bottle'), None)
formatted_bottle_inpaint= {bottle_inpaint['Modèle']: bottle_inpaint['Hyperparamètres']}
print(formatted_bottle_inpaint)

models_hyperparams_bottle = {**formatted_bottle_inpaint, **formatted_bottle_color, **formatted_bottle_mae}

In [ ]:
inpainting_model, colorization_model, mae_model = training_threshold_roc_curves_auroc_confusion(train_loader = bottle_train_loader, 
                                                                                                test_loader = bottle_test_loader,
                                                                                                dataset_name = "Bottle Dataset", 
                                                                                                device='cuda', 
                                                                                                epochs=40, 
                                                                                                models_hyperparams=models_hyperparams_bottle)

In [ ]:
# Visualiser la colorisation sur des images test aléatoires
visualize_reconstructions(inpainting_model, bottle_test_loader, device)
visualize_reconstructions(colorization_model, bottle_test_loader, device)
visualize_reconstructions(mae_model, bottle_test_loader, device)

Enlevé pour mettre le fichier sur le git...

### Capsule Dataset

In [ ]:
# Filtrer pour "capsule" et reformater
capsule_mae = next((entry for entry in extracted_mae if entry['Dataset'] == 'capsule'), None)
formatted_capsule_mae = {capsule_mae['Modèle']: capsule_mae['Hyperparamètres']}
#print(formatted_capsule_mae)

capsule_color = next((entry for entry in extracted_color if entry['Dataset'] == 'capsule'), None)
formatted_capsule_color = {capsule_color['Modèle']: capsule_color['Hyperparamètres']}
#print(formatted_capsule_color)

capsule_inpaint = next((entry for entry in extracted_inpaint if entry['Dataset'] == 'capsule'), None)
formatted_capsule_inpaint= {capsule_inpaint['Modèle']: capsule_inpaint['Hyperparamètres']}
#print(formatted_capsule_inpaint)

models_hyperparams_capsule = {**formatted_capsule_inpaint, **formatted_capsule_color, **formatted_capsule_mae}
print(models_hyperparams_capsule)

In [ ]:
inpainting_model, colorization_model, mae_model = training_threshold_roc_curves_auroc_confusion(train_loader = capsule_train_loader, 
                                                                                                test_loader = capsule_test_loader,
                                                                                                dataset_name = "Capsule Dataset", 
                                                                                                device='cuda', 
                                                                                                epochs=40, 
                                                                                                models_hyperparams=models_hyperparams_capsule)

In [ ]:
# Visualiser la colorisation sur des images test aléatoires
visualize_reconstructions(inpainting_model, capsule_test_loader, device)
visualize_reconstructions(colorization_model, capsule_test_loader, device)
visualize_reconstructions(mae_model, capsule_test_loader, device)

### Toothbrush Dataset

In [ ]:
# Filtrer pour "toothbrush" et reformater
toothbrush_mae = next((entry for entry in extracted_mae if entry['Dataset'] == 'toothbrush'), None)
formatted_toothbrush_mae = {toothbrush_mae['Modèle']: toothbrush_mae['Hyperparamètres']}
#print(formatted_toothbrush_mae)

toothbrush_color = next((entry for entry in extracted_color if entry['Dataset'] == 'toothbrush'), None)
formatted_toothbrush_color = {toothbrush_color['Modèle']: toothbrush_color['Hyperparamètres']}
#print(formatted_toothbrush_color)

toothbrush_inpaint = next((entry for entry in extracted_inpaint if entry['Dataset'] == 'toothbrush'), None)
formatted_toothbrush_inpaint= {toothbrush_inpaint['Modèle']: toothbrush_inpaint['Hyperparamètres']}
#print(formatted_toothbrush_inpaint)

models_hyperparams_toothbrush = {**formatted_toothbrush_inpaint, **formatted_toothbrush_color, **formatted_toothbrush_mae}
print(models_hyperparams_toothbrush)

In [ ]:
inpainting_model, colorization_model, mae_model = training_threshold_roc_curves_auroc_confusion(train_loader = toothbrush_train_loader, 
                                                                                                test_loader = toothbrush_test_loader,
                                                                                                dataset_name = "Toothbrush Dataset", 
                                                                                                device='cuda', 
                                                                                                epochs=40, 
                                                                                                models_hyperparams=models_hyperparams_toothbrush)

In [ ]:
# Visualiser la colorisation sur des images test aléatoires
visualize_reconstructions(inpainting_model, toothbrush_test_loader, device)
visualize_reconstructions(colorization_model, toothbrush_test_loader, device)
visualize_reconstructions(mae_model, toothbrush_test_loader, device)

### Engine Wiring Dataset

In [ ]:
# Filtrer pour "engine_wiring" et reformater
engine_wiring_mae = next((entry for entry in extracted_mae if entry['Dataset'] == 'engine_wiring'), None)
formatted_engine_wiring_mae = {engine_wiring_mae['Modèle']: engine_wiring_mae['Hyperparamètres']}
#print(formatted_engine_wiring_mae)

engine_wiring_color = next((entry for entry in extracted_color if entry['Dataset'] == 'engine_wiring'), None)
formatted_engine_wiring_color = {engine_wiring_color['Modèle']: engine_wiring_color['Hyperparamètres']}
#print(formatted_engine_wiring_color)

engine_wiring_inpaint = next((entry for entry in extracted_inpaint if entry['Dataset'] == 'engine_wiring'), None)
formatted_engine_wiring_inpaint = {engine_wiring_inpaint['Modèle']: engine_wiring_inpaint['Hyperparamètres']}
#print(formatted_engine_wiring_inpaint)

models_hyperparams_engine_wiring = {**formatted_engine_wiring_inpaint, **formatted_engine_wiring_color, **formatted_engine_wiring_mae}
print(models_hyperparams_engine_wiring)

In [ ]:
inpainting_model, colorization_model, mae_model = training_threshold_roc_curves_auroc_confusion(train_loader = engine_wiring_train_loader, 
                                                                                                test_loader = engine_wiring_test_loader,
                                                                                                dataset_name = "Engine Wiring Dataset", 
                                                                                                device='cuda', 
                                                                                                epochs=40, 
                                                                                                models_hyperparams=models_hyperparams_engine_wiring)

In [ ]:
# Visualiser la colorisation sur des images test aléatoires
visualize_reconstructions(inpainting_model, engine_wiring_test_loader, device)
visualize_reconstructions(colorization_model, engine_wiring_test_loader, device)
visualize_reconstructions(mae_model, engine_wiring_test_loader, device)

## Améliorations Futures

1. **Exploration de modèles alternatifs**

D'autres architectures pour a détections d'anomalies pourraient être explorées pour essayer capturer des relations globales plus efficacement en fonction de l'anomalie.

2. **Optimisation des hyperparamètres**

Avec plus de temps et une plus grande puissance, on aurait pu tester nos hyperparamètres sur des plages encore plus grandes lors de notre validation croisée.

3. **Test sur les différents types d'anomalies et entraînement plus long**

Avec plus de temps, plus d'époques pour certains modèles auraient probablement pu améliorer nos résultats et nos reconstructions encore plus. 
Aussi, nous aurions pu essayer de voir si certaines anomalies étaient mieux détectées que d'autres et essayer de fine-tunner nos modèles encore plus pour essayer de capter ces anomalies qui auraient pu passer à travers nos modèles.